## Robust Optimization

For the following LOP,
$$
\begin{align*}
\min \ & \boldsymbol{c}^\top \boldsymbol{x} \\
{\rm s.t.} \ & \boldsymbol{A x} \leq \boldsymbol{b} \\
& \boldsymbol{x \geq 0}
\end{align*}
$$
what if some of the parameters are not accurate? For example, $\boldsymbol c$ is in some range $[\underline{\boldsymbol c}, \bar{\boldsymbol c}]$. 

In this case, we can resort to robust optimization. 

$$
\begin{align*}
\min_{\boldsymbol{x}\geq 0} \ &\max_{\boldsymbol c \in \mathcal{U}_0} \boldsymbol c^\top \boldsymbol x\\
{\rm s.t.}\  & \boldsymbol a_i^\top \boldsymbol{x} \leq b_i && \forall (\boldsymbol a_i, b_i) \in \mathcal{U}_i, i = 1,...,N\\
&\boldsymbol x \geq \boldsymbol 0  
\end{align*}
$$

Let's use the Knapsack problem as an example. 

(For more details of the implementation of robust models in RSOME, please refer to https://xiongpengnus.github.io/rsome/ro_rsome )

### Robust Knapsack problem

* Cost vector, c is deterministic
* Weight vector, a is uncertain

The robust Knapsack problem can be modeled as follows:
$$
\begin{align*}
\min_{\boldsymbol{x}\geq 0} \ &\boldsymbol c^\top \boldsymbol x\\
{\rm s.t.}\  & \sum_{i = 1}^N (\hat{a}_i + \Delta_i z_i)x_i \leq b && \forall \boldsymbol z \in \mathcal{Z}(\Gamma)\\
&x_i \in \{ 0, 1\} && \forall i = 1,...,N   
\end{align*}
$$
where $\mathcal{Z}(\Gamma) = \left\{\boldsymbol{z}~\left|\begin{array}{l} |z_j| \leq 1 \forall j = 1,...,N \\ \sum_{j = 1}^N |z_j| \leq \Gamma \end{array} \right. \right\}$ and $\Gamma$ is a given number. 

In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.special as ss
import pandas as pd

import rsome as rso
from rsome import ro
from rsome import grb_solver as solver

In [2]:
df = pd.read_csv('KnapsackData.csv')
df.head()

,Item,Weight,Value,Deviation
0,P1,80,74,40
1,P2,448,61,224
2,P3,462,56,231
3,P4,158,3,79
4,P5,55,37,28


In [3]:
#capacity
b = 10000
#number of item
N = len(df)
c = df['Value'].values
a_hat = df['Weight'].values
delta = df['Deviation'].values

You can directly solve robust knapsack using RSOME without deriving the robust counterpart.

In [4]:
r=3

m1 = ro.Model('Robust')
x = m1.dvar(N, vtype = "B")
z = m1.rvar(N)  ## rvar stands for random variable

z_set = (abs(z) <= 1, rso.norm(z, 1) <= r) ## define uncertainty set

m1.max(sum([c[i]*x[i] for i in range(N)])) ## alternatively, just use m1.max(c @ x)
m1.st(((a_hat + z*delta) @ x <= b).forall(z_set))
    
m1.solve(solver)

Zr = m1.get()
Xr = x.get()
print(Xr)
print(Zr)

Academic license - for non-commercial use only - expires 2022-07-30
Using license file C:\Users\kings\gurobi.lic
Being solved by Gurobi...
Solution status: 2
Running time: 0.0479s
[ 1.  1. -0. -0.  1. -0.  1.  1.  1. -0. -0.  1.  1.  1.  1. -0. -0. -0.
  1. -0.  1. -0. -0. -0.  1.  1. -0. -0. -0.  1. -0. -0. -0. -0. -0.  1.
 -0. -0.  1. -0. -0. -0.  1. -0.  1. -0. -0. -0.  1. -0.  1.  1. -0. -0.
 -0.  1. -0. -0. -0. -0.  1.  1.  1.  1. -0. -0. -0.  1. -0.  1.  1. -0.
 -0.  1.  1. -0. -0. -0. -0. -0.  1. -0. -0.  1. -0. -0. -0.  1.  1. -0.
 -0.  1.  1. -0. -0.  1. -0. -0.  1.  1.]
2730.0


## Distributionally Robust Optimization 

When the distribution of some random variables are not known but some historical data are available. We can use distributionally robust optimization to model the problem. We use the simplified portfolio optimization as a toy example to demonstrate. 

For a more sophisticated model and more details about the implementation of DRO in RSOME, please refer to https://xiongpengnus.github.io/rsome/example_dro_portfolio and https://xiongpengnus.github.io/rsome/dro_rsome

### Distributionally Robust Portfolio Optimization

For the portfolio optimization problem, suppose we do not know the distribution but can obtain can statistics from historical data, we can model the problem as follows:
$$
\begin{align}
\max\ & \inf_{{\mathbb P}\in \mathcal{F}(\mathbb P)} \mathbb{E}_{\mathbb P}\left[\sum_{i = 1}^N \tilde{r}_i x_i\right]\\
\mbox{s.t.}\ & \boldsymbol{x}^\top \boldsymbol{\Sigma} \boldsymbol{x} \leq \gamma \\
&\boldsymbol{1}^\top \boldsymbol{x} = 1\\
&\boldsymbol{x} \geq \boldsymbol{0}
\end{align}
$$
where 
$$
\mathcal{F}(\mathbb{P}) = \left[
\mathbb{P}~\left|~\begin{array}{l} 
\tilde{\boldsymbol r} \sim \mathbb{P} \\
\mathbb{E}_{\mathbb{P}} [\tilde{\boldsymbol r}] = \boldsymbol \mu \\
\mathbb{E}_{\mathbb{P}} [|\tilde{\boldsymbol r} - \boldsymbol \mu |] \leq \boldsymbol \sigma \\
\mathbb{P}[\underline{\boldsymbol r } \leq \tilde{\boldsymbol r} \leq \bar{\boldsymbol r}] = 1
\end{array}\right.
\right]
$$

Here, $\boldsymbol \mu$ is the expected return, $\boldsymbol \sigma$ is the mean absolute deviation of the return, and $\underline{\boldsymbol r }, \bar{\boldsymbol r}$ are the minimum and maximum return respectively. 

In [5]:
import pandas as pd
# Creating a list of Stock Tickers
from pandas_datareader import data as wb
stocks = ['HSBC','JPM','AAPL','WMT','AMZN','MSFT']
price_data = pd.DataFrame()
# Pulling closing price   
for stock in stocks:
    price_data[stock] = wb.DataReader(stock, data_source = 'yahoo', start = '2010-1-1', end = '2019-12-31')['Adj Close']
num_stocks = len(stocks)

In [6]:
# pf_data pct change
day_returns = price_data.pct_change()
print(day_returns.head(5))

                HSBC       JPM      AAPL       WMT      AMZN      MSFT
Date                                                                  
2009-12-31       NaN       NaN       NaN       NaN       NaN       NaN
2010-01-04  0.021545  0.029554  0.015565  0.014593 -0.004609  0.015420
2010-01-05  0.015947  0.019370  0.001729 -0.009958  0.005900  0.000323
2010-01-06  0.001181  0.005495 -0.015907 -0.002235 -0.018116 -0.006137
2010-01-07 -0.003203  0.019809 -0.001848  0.000560 -0.017013 -0.010400


In [7]:
return_mu = day_returns.mean().values
return_mu

array([0.00014628, 0.00070092, 0.00109278, 0.00047737, 0.00123128,
       0.00085102])

In [8]:
return_max = day_returns.max().values
return_min = day_returns.min().values

return_max, return_min

(array([0.06413007, 0.08438344, 0.08874132, 0.10898378, 0.15745701,
        0.10452254]),
 array([-0.09042417, -0.09414857, -0.12355786, -0.10183239, -0.12656835,
        -0.11399539]))

In [9]:
return_cov = day_returns.cov().values
return_cov

array([[1.79541333e-04, 1.37131464e-04, 7.89104880e-05, 3.84581740e-05,
        9.13261522e-05, 8.71992729e-05],
       [1.37131464e-04, 2.49489807e-04, 9.49789499e-05, 5.11253546e-05,
        1.06920018e-04, 1.06977260e-04],
       [7.89104880e-05, 9.49789499e-05, 2.63065146e-04, 3.97796095e-05,
        1.25714876e-04, 1.06190702e-04],
       [3.84581740e-05, 5.11253546e-05, 3.97796095e-05, 1.18093895e-04,
        4.32417455e-05, 4.77860218e-05],
       [9.13261522e-05, 1.06920018e-04, 1.25714876e-04, 4.32417455e-05,
        3.80680279e-04, 1.36375164e-04],
       [8.71992729e-05, 1.06977260e-04, 1.06190702e-04, 4.77860218e-05,
        1.36375164e-04, 2.04850040e-04]])

In [10]:
return_mad = day_returns.mad().values
return_mad

array([0.0094731 , 0.01118226, 0.01158512, 0.00739665, 0.01341392,
       0.01008124])

In [11]:
N = len(return_mu)

In [12]:
import rsome
from rsome import dro
from rsome import norm
from rsome import E
from rsome import grb_solver as solver

N = len(return_mu)

model = dro.Model()             # create a dro model 
r = model.rvar(N)               # define random return
u = model.rvar(N)               # define a auxiliary random variable u
x = model.dvar(N)

fset = model.ambiguity()        # create an ambiguity set

fset.suppset(r >= return_min, 
             r <= return_max,
             u >= r - return_mu,
             u >= return_mu - r)

fset.exptset(E(r) == return_mu, 
             E(u) <= return_mad )

model.maxinf(E(r @ x), fset)    # maximize the worst-case expectation over fset

model.st(x.sum() == 1,
         rsome.quad(x, return_cov) <= 0.0001,
         x >= 0)       

model.solve(solver)

x.get()

Being solved by Gurobi...
Solution status: 2
Running time: 0.0118s


array([3.74369264e-06, 5.83241404e-02, 2.52571458e-01, 3.88211098e-01,
       1.54102693e-01, 1.46786869e-01])

### Distributionally robust multi-item newsvendor problem (Optional)

Now suppose there are $N$ products and $K$ historical samples. Specifically, 
* $N$ items to sell. 
* For each item $i$, unit cost is $𝑐_𝑖$, selling price $𝑝_𝑖$. 
* Order quantity $𝑥_𝑖, i = 1, … , N$. 
* Assume salvage price is zero.
* Each item $i$ incurs storage space $𝑠_𝑖$. Total space available is $C$. 
* $K$ instances of demands available. 
* Let $𝑑_𝑘𝑖$ denotes the demand of item $i$ in instance $k, k = 1, …, K$. 

For this problem, it's difficult to obtain the demand distribution. We can use historical data to estimate the distribution, but most likely, we won't have enough data! For example, suppose we have 10 products and each product has 3 possible values. We then will need at least 
$$
3^{10} = 59049 
$$
samples!

If we collect one sample each day, it will take us at least 161.77 years to obtain all the possible demands!!

However, we can derive some statistics such as support, mean, and mean standard deviation of the demand from historical data so that we can formulate the multi-item newsvendor problem under ambiguity as follows: 

$$
\begin{align*}
\max_{\boldsymbol{x}\geq 0} \ &\min_{\mathbb{P} \in \mathcal{F}(\mathbb{P})}\mathbb{E}_{\mathbb{P}} \sum_{i = 1}^N\left( p_i \min\{ x_i, \tilde{d_i} \}  \right) - \boldsymbol{c}^\top\boldsymbol{x} \\
\end{align*}
$$
where 
$$
\mathcal{F}(\mathbb{P}) = \left[
\mathbb{P}~\left|~\begin{array}{l} 
\tilde{\boldsymbol d} \sim \mathbb{P} \\
\mathbb{E}_{\mathbb{P}} [\tilde{\boldsymbol d}] = \boldsymbol \mu \\
\mathbb{E}_{\mathbb{P}} [|\tilde{\boldsymbol d} - \boldsymbol \mu |] \leq \boldsymbol \sigma \\
\mathbb{P}[\underline{\boldsymbol d } \leq \tilde{\boldsymbol d} \leq \bar{\boldsymbol d}] = 1
\end{array}\right.
\right]
$$

This model can be reformulated as
$$
\begin{align*}
\max_{\boldsymbol{x}\geq 0} \ &\min_{\mathbb{P} \in \mathcal{F}(\mathbb{P})}\mathbb{E}_{\mathbb{P}} [\boldsymbol p^\top \boldsymbol y(\tilde{\boldsymbol d})] - \boldsymbol{c}^\top\boldsymbol{x} \\
{\rm s.t.}\  & \boldsymbol y(\boldsymbol d) \leq \boldsymbol x && \forall \boldsymbol d \in [\underline{\boldsymbol d }, \bar{\boldsymbol d}]\\
&\boldsymbol y(\boldsymbol d) \leq \boldsymbol d && \forall \boldsymbol d \in [\underline{\boldsymbol d }, \bar{\boldsymbol d}]\\  
&\boldsymbol s^\top \boldsymbol x \leq C \\
&\boldsymbol x \geq \boldsymbol 0  
\end{align*}
$$

We can solve this model as follows (A more sophisticated model with Wasserstein ambiguity set can be found here: https://xiongpengnus.github.io/rsome/example_dro_nv):

In [13]:
def Multi_Newsvendor_DRO(p,c,D,s,C):
    K,N = D.shape
    D_mu = D.mean().values
    D_mad = D.mad().values
    D_min = D.min().values
    D_max = D.max().values
    
    mnv = dro.Model('Multi_Newsvendor')
    
    x = mnv.dvar(N)
    y = mnv.dvar(N)

    d = mnv.rvar(N)
    u = mnv.rvar(N)       
    
    y.adapt(d)
    y.adapt(u)
    
    fset = mnv.ambiguity()        # create an ambiguity set
    fset.suppset(d >= D_min, 
                 d <= D_max,
                 u >= d - D_mu,
                 u >= D_mu - d)

    fset.exptset(E(d) == D_mu, 
                 E(u) <= D_mad )
    
    mnv.maxinf( E(p @ y) - c @ x, fset )
    
    mnv.st( y <= x )
    mnv.st( y <= d )
    mnv.st( s @ x <= C )
    mnv.st( x >= 0)
    
    mnv.solve(solver, display = False)
    return x.get(), mnv.get()

In [14]:
Demand_data = pd.read_csv("Demand_data.csv")
Demand_data.head()

,real_date,prod_1,prod_2,prod_3,prod_4,prod_5,prod_6,prod_7,prod_8,prod_9,prod_10,prod_11,prod_12,prod_13,prod_14,prod_15
0,1/2/2013,99,59,348,47,37,422,67,132,209,28,99,113,70,80,404
1,1/3/2013,80,41,180,34,24,329,47,106,162,27,80,65,49,54,323
2,1/4/2013,73,41,141,33,19,257,55,94,124,26,61,59,36,57,310
3,1/5/2013,103,60,259,38,28,359,66,148,194,28,86,83,66,68,467
4,1/6/2013,109,70,439,46,37,376,63,150,252,33,124,114,77,84,579


In [15]:
from sklearn.model_selection import train_test_split

demand_train, demand_test = train_test_split(Demand_data, test_size=0.3)
Demand = demand_train.drop('real_date', axis=1, inplace=False)

In [16]:
Param_data = pd.read_csv("Param_data.csv")
Param_data

,Unnamed: 0,prod_1,prod_2,prod_3,prod_4,prod_5,prod_6,prod_7,prod_8,prod_9,prod_10,prod_11,prod_12,prod_13,prod_14,prod_15
0,Price,10.6,7.3,10.7,9.5,7.6,14.8,10.4,4.7,14.0,6.0,6.1,14.8,11.9,6.9,8.6
1,Cost,3.2,3.9,5.3,3.8,2.1,6.9,5.2,2.9,4.6,2.4,2.4,5.9,3.8,1.7,2.5
2,Space,0.4,0.1,0.4,0.4,0.3,0.3,0.3,0.4,0.4,0.3,0.4,0.1,0.1,0.5,0.2


In [17]:
price = np.array( Param_data.values[0,1:16].astype(np.float64) )
cost = np.array( Param_data.values[1,1:16].astype(np.float64) )
space = np.array( Param_data.values[2,1:16].astype(np.float64) )
total_space = 300

In [18]:
order_dro, profit_dro = Multi_Newsvendor_DRO(price,cost,Demand,space,total_space)
print('The optimal order quantity is:', order_dro )
print('-------------------------------------')
print('And the corresponding worst-case expected profit is:', profit_dro )

The optimal order quantity is: [ 61.00340426  53.18553191  65.57978723  43.31404255  33.38468085
 239.72170213  67.77021277   0.         110.72255319  32.18382979
  11.          55.45106383  70.03574468  13.         235.33361702]
-------------------------------------
And the corresponding worst-case expected profit is: 5626.084876892422


#### Solve the problem empirically

Alternatively, as we have covered last week, we can solve the problem empirically:

In [19]:
def Multi_Newsvendor_Emp(p,c,D,s,C):
    K,N = D.shape
    
    mnv = ro.Model('Multi_Newsvendor')
    x = mnv.dvar(N)
    t = mnv.dvar( (K,N) )
    
    mnv.max( 1/K*( (t @ p).sum() ) - c @ x )
    
    mnv.st( t[:,i] <= x[i] for i in range(N) )
    mnv.st( t <= D )
    mnv.st( s @ x <= C )
    mnv.st( x >= 0)
    
    mnv.solve(solver, display = False)
    return x.get(), mnv.get()

In [20]:
price = np.array( Param_data.values[0,1:16].astype(np.float64) )
cost = np.array( Param_data.values[1,1:16].astype(np.float64) )
space = np.array( Param_data.values[2,1:16].astype(np.float64) )
total_space = 300

In [21]:
order_emp, profit_emp = Multi_Newsvendor_Emp(price,cost,Demand.values,space,total_space)
print('The optimal order quantity is:', order_emp )
print('-------------------------------------')
print('And the corresponding expected profit is:', profit_emp )

The optimal order quantity is: [ 54.    41.    72.    34.    29.   207.    48.     0.    92.    25.
  74.25  55.    73.    27.   232.  ]
-------------------------------------
And the corresponding expected profit is: 5944.017085105508


#### Evaluate the two solutions

In [25]:
## Optimal order quantity (theoretical)
Demand_test = np.array( demand_test.values[:,1:16].astype(int) )

profit_dro = np.mean( np.minimum(order_dro @ price, Demand_test @ price) - cost @ order_dro )
std_dro = np.std( np.minimum(order_dro @ price, Demand_test @ price) - cost @ order_dro )

print( profit_dro,std_dro )

profit_emp = np.mean( np.minimum(price @ order_emp, Demand_test @ price) - cost @ order_emp )
std_emp = np.std( np.minimum(price @ order_emp, Demand_test @ price) - cost @ order_emp )

print( profit_emp, std_emp )

Profit_imp = (profit_dro - profit_emp)/profit_emp*100
Std_imp = (std_dro - std_emp)/std_emp*100

print( Profit_imp, Std_imp )

6854.927599164134 983.490771727705
6693.11820436508 723.1880735513305
2.4175487397417648 35.9937764042646


We can see that DRO improves the profit by 2.68 percent while the standard deviation is 36 percent higher!